In [3]:
!pip install kaggle

     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     ---------------------------------------- 82.1/82.1 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105134 sha256=e55c7a5564523c9d7c98c17b9d2cd30aba886bad3d0a08a2a08bd6eb99471a0b
  Stored in directory: c:\users\abhinav pandey\appdata\local\pip\cache\wheels\ec\d9\04\93be57edf3835182a8e76d8ff87f7e77a0113f473ba3721ee0
Successfully built kaggle


In [6]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection- Kaggle API**

In [8]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [9]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [10]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [46]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0




  0%|          | 0.00/66.0M [00:00<?, ?B/s]
  2%|1         | 1.00M/66.0M [00:01<01:34, 720kB/s]
  3%|3         | 2.00M/66.0M [00:01<00:46, 1.43MB/s]
  5%|4         | 3.00M/66.0M [00:02<00:35, 1.85MB/s]
  6%|6         | 4.00M/66.0M [00:02<00:26, 2.48MB/s]
  8%|7         | 5.00M/66.0M [00:02<00:22, 2.83MB/s]
  9%|9         | 6.00M/66.0M [00:02<00:20, 3.13MB/s]
 11%|#         | 7.00M/66.0M [00:03<00:18, 3.32MB/s]
 12%|#2        | 8.00M/66.0M [00:03<00:17, 3.47MB/s]
 14%|#3        | 9.00M/66.0M [00:03<00:16, 3.58MB/s]
 15%|#5        | 10.0M/66.0M [00:03<00:16, 3.66MB/s]
 17%|#6        | 11.0M/66.0M [00:04<00:15, 3.71MB/s]
 18%|#8        | 12.0M/66.0M [00:04<00:15, 3.75MB/s]
 20%|#9        | 13.0M/66.0M [00:04<00:14, 3.78MB/s]
 21%|##1       | 14.0M/66.0M [00:04<00:14, 3.79MB/s]
 23%|##2       | 15.0M/66.0M [00:05<00:13, 3.82MB/s]
 24%|##4       | 16.0M/66.0M [00:05<00:14, 3.57MB/s]
 26%|##5       | 17.0M/66.0M [00:05<00:13, 3.83MB/s]
 27%|##7       | 18.0M/66.0M [00:06<00:13, 3.64MB/s]
 2

In [12]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [15]:
data = pd.read_csv("IMDB Dataset.csv")

In [16]:
data.shape

(50000, 2)

In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [19]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [20]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [22]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [23]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [24]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [25]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [26]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [27]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [28]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [29]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [38]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [37]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [39]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the Model**

In [40]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 173s 338ms/step - accuracy: 0.7368 - loss: 0.5150 - val_accuracy: 0.8460 - val_loss: 0.3639
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 181s 361ms/step - accuracy: 0.8458 - loss: 0.3653 - val_accuracy: 0.8461 - val_loss: 0.3584
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 246s 492ms/step - accuracy: 0.8666 - loss: 0.3259 - val_accuracy: 0.8559 - val_loss: 0.3431
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 169s 338ms/step - accuracy: 0.8862 - loss: 0.2852 - val_accuracy: 0.8602 - val_loss: 0.3375
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 396ms/step - accuracy: 0.8893 - loss: 0.2782 - val_accuracy: 0.8733 - val_loss: 0.3131


**Model Evaluation**

In [41]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 71ms/step - accuracy: 0.8781 - loss: 0.2980
Test Loss: 0.29703667759895325
Test Accuracy: 0.8795999884605408


**Building a Predictive System**

In [42]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [43]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
The sentiment of the review is: positive


In [44]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
The sentiment of the review is: negative


In [45]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
The sentiment of the review is: negative
